<a href="https://colab.research.google.com/github/Omars2003/HuggingFace/blob/main/Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install transformers[sentencepiece]



In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DefaultDataCollator
dataset = load_dataset("squad")

In [ ]:
def muestra_ejemplos(dataset, num_samples=4, seed=50):
  ejemplos = dataset["train"].shuffle(seed=seed).select(range(num_samples))
  for example in ejemplos: #aqui hacemos un arreglo para que imprima 4 ejemplos de manera random
        print(f"\n'>> Pregunta: {example['question']}'") #aqui imprime el titulo
        print(f"'>> Respuesta: {example['context']}'")#aqui imprime la reseña como tal


muestra_ejemplos(dataset)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [6]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

         # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs
   



In [7]:
tokenized_squad = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

  0%|          | 0/88 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

In [8]:
data_collator = DefaultDataCollator(return_tensors="tf")


In [9]:
tf_train_set = tokenized_squad["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "start_positions", "end_positions"],

    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = tokenized_squad["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "start_positions", "end_positions"],

    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [10]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 2
total_train_steps = (len(tokenized_squad["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

In [ ]:
from transformers import TFAutoModelForQuestionAnswering
model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")


In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)

In [13]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=4)

Epoch 1/4
5475/5475 [==============================] - 2330s 424ms/step - loss: 1.4817 - val_loss: 1.1364
Epoch 2/4
5475/5475 [==============================] - 2316s 423ms/step - loss: 0.9621 - val_loss: 1.1012
Epoch 3/4
5475/5475 [==============================] - 2318s 423ms/step - loss: 0.8471 - val_loss: 1.1012
Epoch 4/4
5475/5475 [==============================] - 2319s 423ms/step - loss: 0.8476 - val_loss: 1.1012
